<a href="https://colab.research.google.com/github/YosukeSugiura/ImageQualityAssesment/blob/master/IQA_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nnabla-ext-cuda100
!git clone https://github.com/YosukeSugiura/ImageQualityAssesment.git


     |████████████████████████████████| 18.6MB 1.6MB/s 
     |████████████████████████████████| 13.2MB 31.5MB/s 
     |████████████████████████████████| 7.1MB 42.0MB/s 
Cloning into 'ImageQualityAssesment'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [3]:
cd ImageQualityAssesment

/content/ImageQualityAssesment


In [0]:
from __future__ import absolute_import
from six.moves import range

import os
import time
import numpy as np

#   NNabla
import nnabla as nn
import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
import nnabla.initializer as I
from nnabla.ext_utils import get_extension_context  # GPU

#   Google Colab
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow

#   Image
import cv2
from IPython.display import display, Javascript
from base64 import b64decode

#   Originals
from settings import settings
import data as dt


In [0]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
      return video
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
# -------------------------------------------
#   Network for IQA
# -------------------------------------------
def network(input, scope="", test=False):
    """
    Define Convolutional DNN
       input:   input image ( size must be 64 by 64 )
       test:    test or not
    """

    with nn.parameter_scope(scope):
        c1 = PF.prelu(PF.convolution(input, 10, (8,8), (2,2), (2,2), name='cnv1'))   # Convolution : 64,64,3 -> 2,32,29
        c2 = F.max_pooling(c1, (2,2), (2,2))                                 # Max Pooling : 2,32,29 -> 2,32,14
        #c2 = PF.batch_normalization(c2, batch_stat=not test, name='bn1')  # Batch Normalization
        c4 = PF.prelu(PF.convolution(c2, 16, (8,8), (2,2), (1,1), name='cnv2'))          # Convolution : 2,32,14 -> 16,15,5
        #c4 = PF.batch_normalization(c4, batch_stat=not test, name='bn2')  # Batch Normalization
        c6 = F.max_pooling(c4, (2,2), (2,2))                                 # Max Pooling : 16,15,5 -> 16,15,1
        #c6 = PF.batch_normalization(c5, batch_stat=not test)                        # Batch Normalization
        c7 = PF.prelu(c6, name='PReLU')                                             # PReLU
        c8 = PF.affine(c7, (100,), name='Affine')                                   # Affine : 16,15,1 -> 142
        c9 = F.relu(c8)                                                     # ReLU
        c10= PF.affine(c9, (1,), name='Affine2')                                    # Affine : 142 -> 1

    return c10

In [0]:
##  ~~~~~~~~~~~~~~~~~~~
##   Demo
##  ~~~~~~~~~~~~~~~~~~~


args = settings()

#   Input Variable
nn.clear_parameters()                           #   Clear
Input   = nn.Variable([1, 3, 64, 64])#   Input

#   Network Definition
Name    = "CNN"                                 #   Name of scope which includes network models (arbitrary)
Output  = network(Input, scope=Name, test=True) #   Network & Output

#   Load data
with nn.parameter_scope(Name):
    nn.load_parameters(os.path.join(args.model_save_path, "network_param_{:04}.h5".format(args.epoch)))


In [0]:
#   Capture Image
while (True):
  filename = take_photo()
  display(Image(filename))
  I = cv2.imread(filename)
    
  Img_resize = cv2.resize(I, (64, 64)).transpose(2, 0, 1)  # Resize (*,*,3) -> (3,64,64)
  Input.d = np.expand_dims(Img_resize, axis=0)
  Output.forward()
  score = np.max([min([Output.d[0][0] / 9 * 100, 100]), 0] )

  print('IQA Score :  {}'.format(score))